# Single-Qubit Gate Operators Optimization

## Part 1

### Given Input

A string in the form of a comma-separated list of sequential rotations on one particular qubit.
The string takes the form: “{Axis}({Angle}}, {Axis}({Angle}), …” where {Axis} can be X or Y and angle is an integer specifying the angle of rotation in degrees. 

### Required Output

Generate a list of sequential rotations of gates $X$ and $Y$ to perform on the qubit optimised to perform as few individual rotations as possible.


## Solution

Let's first import all the packages that we will need during this task.

In [211]:
import numpy as np
import random

Here, we will define the function for $R_x$, $R_y$ and $R_z$. The expression of each Pauli rotations are as follows
\begin{align}
R_{x}\left(\theta\right)&=\cos\left(\frac{\theta}{2}\right)\begin{bmatrix}
1&0\\
0&1
\end{bmatrix}
-\dot{\iota}\sin\left(\frac{\theta}{2}\right) \begin{bmatrix}
0&1\\
1&0
\end{bmatrix}=\begin{bmatrix}
\cos\frac{\theta}{2}&-\dot{\iota} \sin\frac{\theta}{2}\\
-\dot{\iota} \sin\frac{\theta}{2}&\cos\frac{\theta}{2}
\end{bmatrix},\\
R_{y}\left(\theta\right)&=\cos\left(\frac{\theta}{2}\right)\begin{bmatrix}
1&0\\
0&1
\end{bmatrix}
-\dot{\iota}\sin\left(\frac{\theta}{2}\right) \begin{bmatrix}
0&-\dot{\iota}\\
\dot{\iota}&0
\end{bmatrix} =\begin{bmatrix}
\cos\frac{\theta}{2}&-\sin\frac{\theta}{2}\\
 \sin\frac{\theta}{2}&\cos\frac{\theta}{2}
\end{bmatrix},\\
R_{z}\left(\theta\right)&=\cos\left(\frac{\theta}{2}\right)\begin{bmatrix}
1&0\\
0&1
\end{bmatrix}
-\dot{\iota}\sin\left(\frac{\theta}{2}\right) \begin{bmatrix}
1&0\\
0&-1
\end{bmatrix}=\begin{bmatrix}
e^{-\dot{\iota}\frac{\theta}{2}}&0\\
 0&e^{\dot{\iota}\frac{\theta}{2}}
\end{bmatrix}.
\end{align}

Each function takes input in degrees to generate output.

In [257]:
def Rx(theta):
    I=np.array([[1,0],[0,1]])
    X=np.array([[0,1],[1,0]])
    return (np.cos(theta*np.pi/(2*180))*I-1j*np.sin(theta*np.pi/(2*180))*X)
def Ry(theta):
    I=np.array([[1,0],[0,1]])
    Y=np.array([[0,-1j],[1j,0]])
    return (np.cos(theta*np.pi/(2*180))*I-1j*np.sin(theta*np.pi/(2*180))*Y)
def Rz(theta):
    I=np.array([[1,0],[0,1]])
    Z=np.array([[1,0],[0,-1]])
    return (np.cos(theta*np.pi/(2*180))*I-1j*np.sin(theta*np.pi/(2*180))*Z)

This block of code is randomly generating gate sequence of $X(\theta_1),Y(\theta_2)\cdots$  by user specified length or we can input by ourself. 

In [309]:
k = ''
while k not in ['y', 'n', 'Y', 'N']:
    k = input('Press y to enter sequence manually or n to generate sequence automatically: ')

if k in ['y', 'Y']:
    q = input('Enter sequence of comma separated {Axis}({Angle}): ')
    f = ''
    
    for l in q.split(','):
        f = f'{f}{l}, '  
        if l.split('(')[0] not in ['X', 'Y']:
            raise Exception('Only X and Y allowed')
            
        if l.split('(')[1][:-1].replace('.','',1).isnumeric() == False:
            raise Exception('Only numbers allowed')
    
    f = f'"{f[:-2]}"'

else:
    start, end, f , foo= 0, 360,"" , ['X', 'Y'] 
    print('Enter length of sequence:')
    l = input()
    l=int(l)

    for _ in range(l):
      x=random.choice(foo)
      y = random.randint(start, end)
      z=[f,x,'(',str(y),')',', ']
      f = "".join(z)
    f=f[:-2]
    f=['"',f,'"']
    f = "".join(f)


Press y to enter sequence manually or n to generate sequence automatically: y
Enter sequence of comma separated {Axis}({Angle}): X(33.3),Y(0.57)


This block of the code combine the all the input sequence to get one single unitary operator.

In [310]:
f=f[1:-1]
f=f.split(', ')
U=np.array([[1,0],[0,1]])
for i in range(len(f)):
    if f[i][0]=='X':
        U=Rx(float(f[i][2:-1]))@U
    else:
        U=Ry(float(f[i][2:-1]))@U

In order to minimize the gate sequence, we can decompose the single qubit unitary matrix into general representation of rotation operators $R_x(\alpha)R_y(\theta)R_x(\beta)$. Simplifying the expression gives us
\begin{align}
R_x(\alpha)R_y(\theta)R_x(\beta)=
\begin{bmatrix}
\cos\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2}+\dot{\iota}\sin\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2}&-\cos\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2}-\dot{\iota}\sin\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2}\\
\cos\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2}-\dot{\iota}\sin\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2}&\cos\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2}-\dot{\iota}\sin\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2}
\end{bmatrix}.
\end{align}

The elemenets of the unitary matrix can be rewritten as a complex number. 
\begin{align}
U=\begin{bmatrix}
x+\dot{\iota}y& p+\dot{\iota}q \\
-(p+\dot{\iota}q)^{*}&x-\dot{\iota}y
\end{bmatrix}.
\end{align}
By equating real and complex elements of  $R_x(\alpha)R_y(\theta)R_x(\beta)$ with $U$ leads to
\begin{align}
x&=\cos\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2},\hspace{1cm} y=\sin\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2} \\
p&=-\cos\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2},\hspace{1cm} q=-sin\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2}.
\end{align}

Using basic trigonometry, the angles are calculated as
\begin{align}
\alpha &=\tan^{-1}\left(\frac{-q}{x}\right)+\tan^{-1}\left(\frac{y}{p}\right)\\
\beta &=\tan^{-1}\left(\frac{-q}{x}\right)-\tan^{-1}\left(\frac{y}{p}\right) \\
\theta &=2 \tan^{-1}\left(\frac{-p \cos\left(\frac{\alpha+\beta}{2}\right)}{x\cos\left(\frac{\alpha-\beta}{2}\right)}\right).
\end{align}
It shows that any arbitrary length of $X(\theta_1)Y(\theta_2)\cdots$ gate operations can be implemented into $3$ basic rotations operators of the form $R_x(\alpha)R_y(\theta)R_x(\beta)$.

In [311]:
x,y,p,q=np.real(U[0,0]),np.imag(U[0,0]),np.real(U[0,1]),np.imag(U[0,1])

if (x==0):
    x=10**(-9)
if (p==0):
    p=10**(-9)
alpha=np.arctan(-q/x)+np.arctan(y/p)
beta=np.arctan(-q/x)-np.arctan(y/p)
theta=2*np.arctan((-p/x)*(np.cos((alpha+beta)/2)/np.cos((alpha-beta)/2)))


To prove that the $U_2=U_1$, it must hold the following property of unitary operators
\begin{align}
U_{1}U_{2}^{\dagger}=I.
\end{align}
The code below outputs the identity operator which proves that $U_2=U_1$.

In [312]:
U2=Rx(alpha*180/np.pi)@Ry(theta*180/np.pi)@Rx(beta*180/np.pi)
U1=np.matrix(U)
print(np.round(U2@U1.getH(),7)) 

[[ 1.-0.j -0.+0.j]
 [ 0.+0.j  1.+0.j]]


This block of the code outputs $\alpha$, $\theta$ and $\beta$ in the require form of output.

In [285]:
def print_seq(alpha,beta,theta,str_decomposition):
    alpha=alpha*180/np.pi
    theta=theta*180/np.pi
    beta=beta*180/np.pi
    z=[str_decomposition[0],'(',str(alpha),')',', ',str_decomposition[1],'(',str(theta),')',', ',str_decomposition[2],'(',str(beta),')',', ']
    f = "".join(z)
    f=f[:-2]
    f=['"',f,'"']
    f = "".join(f)
    f=f[1:-1]
    f=f.split(', ')
   
    if alpha in {0,180,360} :
      del(f[0])
    if beta in {0,180,360}:
      del(f[-1])
    if theta in {0,180,360}:
      f=['X','(',str(alpha+beta),')']
      f = "".join(f)
    else:
      f = ", ".join(f)

    f=['"',f,'"']
    f = "".join(f)
    
    return print(f)
print_seq(alpha,beta,theta,'XYX')

"Y(130.0)"


# Part 2

### Given Input

The input sequence as previously specified is given. However, this program will be run on a computer that does not support Y rotations.

### Required Output

Generate a list of sequential rotations of gates $X$ and $Z$ to perform on the qubit optimised to perform as few individual rotations as possible.

## Solution

In the experiment, we do not have the $Y\left(\theta\right)$ available. The simple solution to this problem is to replace the $Y\left(\theta\right)$ gate with the other basic gates. The following relation is useful if we have only $X$ and $Z$ gates are available
\begin{align}
Y\left(\theta\right)=Z\left(90\right)X\left(\theta\right)Z\left(-90\right)
\end{align}


In [286]:
U3=Rx(alpha*180/np.pi)@Rz(90)@Rx(theta*180/np.pi)@Rz(-90)@Rx(beta*180/np.pi)
print(np.round(U3@U1.getH(),7))

[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


In the above code, we utilize $5$ gates to implement the unitary matrix. To further decompose into smaller gates, we can decompose the unitary matrix into  $R_z(\alpha)R_y(\theta)R_z(\beta)$. Simplifying the expression gives us
\begin{align}
R_z(\alpha)R_y(\theta)R_z(\beta)=
\begin{bmatrix}
e^{-\dot{\iota} \left(\frac{\alpha+\beta}{2}\right)} \cos\frac{\theta}{2}&-e^{-\dot{\iota} \left(\frac{\alpha-\beta}{2}\right)}\sin\frac{\theta}{2}\\
e^{\dot{\iota} \left(\frac{\alpha-\beta}{2}\right)}\sin\frac{\theta}{2}&e^{\dot{\iota} \left(\frac{\alpha+\beta}{2}\right)} \cos\frac{\theta}{2}
\end{bmatrix}.
\end{align}

By equating real and complex elements of  $R_x(\alpha)R_y(\theta)R_x(\beta)$ with $U$ leads to
\begin{align}
x&=\cos\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2},\hspace{1cm} y=-\sin\left(\frac{\alpha-\beta}{2}\right) \cos\frac{\theta}{2} \\
p&=\cos\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2},\hspace{1cm} q=-sin\left(\frac{\alpha+\beta}{2}\right) \sin\frac{\theta}{2}.
\end{align}

Using basic trigonometry, the angles are calculated as
\begin{align}
\alpha &=\tan^{-1}\left(\frac{-y}{x}\right)+\tan^{-1}\left(\frac{-q}{p}\right)\\
\beta &=\tan^{-1}\left(\frac{-y}{x}\right)-\tan^{-1}\left(\frac{-q}{p}\right) \\
\theta &=2 \tan^{-1}\left(\sqrt{\frac{p^{2}+q^{2}}{x^{2}+y^{2}}}\right).
\end{align}
It shows that any arbitrary length of $X(\theta_1)Y(\theta_2)\cdots$ gate operations can be implemented into $3$ basic rotations operators of the form $R_z(\alpha)R_y(\theta)R_z(\beta)$.

In [287]:
x,y,p,q=np.real(U[0,0]),np.imag(U[0,0]),np.real(U[0,1]),np.imag(U[0,1])

if (x==0):
    x=10**(-7)
if (p==0):
    p=10**(-7)
alpha=np.arctan(-y/x)+np.arctan(-q/p)
beta=np.arctan(-y/x)-np.arctan(-q/p)
theta_p=2*np.arctan(np.sqrt((p**2 +q**2)/(x**2 +y**2)))  
theta_m=2*np.arctan(-np.sqrt((p**2 +q**2)/(x**2 +y**2)))  

The relationship can be written as
\begin{align}
R_z(\alpha)R_y(\theta)R_z(\beta)&=R_z(\alpha)R_z(90)R_x(\theta)R_z(-90)R_z(\beta)\\
&=R_z(\alpha+90)R_x(\theta)R_z(\beta-90)
\end{align}

We obtain the two different values of $\theta$. We use the value which gives the identity with actual unitary. For the verification of our unitary

In [290]:
U4_p=Rz(alpha*180/np.pi+90)@Rx(theta_p*180/np.pi)@Rz(beta*180/np.pi-90 )
U4_m=Rz(alpha*180/np.pi+90)@Rx(theta_m*180/np.pi)@Rz(beta*180/np.pi-90 )
print(np.round(U4_p@U1.getH(),6))
print(np.round(U4_m@U1.getH(),6))

[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
[[-0.642788+0.j  0.766044+0.j]
 [-0.766044+0.j -0.642788+0.j]]


This block of the code outputs $\alpha$, $\theta$ and $\beta$ in the require form of output.

In [291]:
print_seq(alpha,beta,theta,'ZXZ')

"X(130.0)"


# Part 3

### Given Input

Configurable parameters which takes different nanoseconds to complete

### Required Output

Optimise the instruction sequence such that the total time taken is minimised

## Solution

In the experiment, if the gates are time dependent,  then we select the decomposition depending on time taken by the gates length. For this purpose, we will require $R_x(\alpha)R_z(\theta)R_x(\beta)$ decomposition.  Simplifying the expression gives us
\begin{align}
R_x(\alpha)R_z(\theta)R_x(\beta)=
\begin{bmatrix}
\cos\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2}+\dot{\iota}\cos\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2}&-\sin\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2}-\dot{\iota}\sin\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2}\\
\sin\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2}-\dot{\iota}\sin\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2}&\cos\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2}-\dot{\iota}\cos\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2}
\end{bmatrix}.
\end{align}

By equating real and complex elements of  $R_x(\alpha)R_y(\theta)R_x(\beta)$ with $U$ leads to
\begin{align}
x&=\cos\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2},\hspace{1cm} y=\cos\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2} \\
p&=-\sin\left(\frac{\alpha-\beta}{2}\right) \sin\frac{\theta}{2},\hspace{1cm} q=-sin\left(\frac{\alpha+\beta}{2}\right) \cos\frac{\theta}{2}.
\end{align}

Using basic trigonometry, the angles are calculated as
\begin{align}
\alpha &=\tan^{-1}\left(\frac{-q}{x}\right)+\tan^{-1}\left(\frac{p}{y}\right)\\
\beta &=\tan^{-1}\left(\frac{-q}{x}\right)-\tan^{-1}\left(\frac{-p}{y}\right) \\
\theta &=2 \tan^{-1}\left(\frac{p \sin\left(\frac{\alpha-\beta}{2}\right)}{x\cos\left(\frac{\alpha+\beta}{2}\right)}\right).
\end{align}
It shows that any arbitrary length of $X(\theta_1)Y(\theta_2)\cdots$ gate operations can be implemented into $3$ basic rotations operators of the form $R_x(\alpha)R_z(\theta)R_x(\beta)$.

This block of the sequence asks the user to input the timing of $X$ and $Z$ gate. Based on timing, the program selects the optimized sequence control.

In [308]:
x,y,p,q=np.real(U[0,0]),np.imag(U[0,0]),np.real(U[0,1]),np.imag(U[0,1])

A = input('Enter timing  of Gate Z:')
B = input('Enter timing of Gate X:')

if A.replace('.','',1).isnumeric() and B.replace('.','',1).isnumeric():
    
    lengthz=float(A)
    lengthx=float(B)
    
    if lengthz<lengthx:
        if (x==0):
            x=10**(-7)
        if (p==0):
            p=10**(-7)
        alpha=np.arctan(-y/x)+np.arctan(-q/p)
        beta=np.arctan(-y/x)-np.arctan(-q/p)
        theta_p=2*np.arctan(np.sqrt((p**2 +q**2)/(x**2 +y**2)))  
        theta_m=2*np.arctan(-np.sqrt((p**2 +q**2)/(x**2 +y**2)))  
        U4_p=Rz(alpha*180/np.pi+90)@Rx(theta_p*180/np.pi)@Rz(beta*180/np.pi-90 )
        U4_m=Rz(alpha*180/np.pi+90)@Rx(theta_m*180/np.pi)@Rz(beta*180/np.pi-90 )
        if(np.eye(2) in abs(np.real(np.round(U4_m@U1.getH(),7)))):
            theta= theta_m
        else:
            theta= theta_p
        print('Optimize sequence of gate')    
        print_seq(alpha,beta,theta,'ZXZ')
    else:
        if (y==0):
            y=10**(-8)
        if (x==0):
            x=10**(-8)   
        alpha=np.arctan(-q/x)+np.arctan(-p/y)
        beta=np.arctan(-q/x)-np.arctan(-p/y)
        theta=2*np.arctan((p/x)*(np.sin((alpha-beta)/2)/np.cos((alpha+beta)/2))) 
        U5=Rx(alpha*180/np.pi)@Rz(theta*180/np.pi)@Rx(beta*180/np.pi)
        print('Optimize sequence of gate')
        print_seq(alpha,beta,theta,'XZX')
else:
    raise Exception('A or B is not a number')

Enter timing  of Gate Z:0.4
Enter timing of Gate X:0.3
Optimize sequence of gate
"X(-48.0), Z(83.21235989851225), X(-105.00000000000003)"
